In [3]:
pip show tensorflow

Name: tensorflow
Version: 2.19.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: C:\Users\hp\anaconda3\Lib\site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, termcolor, typing-extensions, wrapt
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [4]:
# System and Data
import os
import numpy as np
import matplotlib.pyplot as plt

# TensorFlow and Keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception, VGG16, ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Sklearn
from sklearn.metrics import classification_report, confusion_matrix


In [5]:
DATASET_PATH = r'C:\Users\hp\Documents\Data\Original'  # Change this if your folder has a different name
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 15


In [6]:
DATASET_PATH = r"C:\Users\hp\Documents\Data\archive"
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

test_val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Optional: use a separate test folder
# test_generator = test_val_datagen.flow_from_directory(
#     'test_dataset/',
#     target_size=IMAGE_SIZE,
#     batch_size=BATCH_SIZE,
#     class_mode='categorical',
#     shuffle=False
# )


Found 10918 images belonging to 5 classes.
Found 2727 images belonging to 5 classes.


In [7]:
def build_model(base_model_class, input_shape=(224, 224, 3), n_classes=3):
    base_model = base_model_class(include_top=False, weights='imagenet', input_shape=input_shape)
    base_model.trainable = False  # Freeze all layers

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    output = Dense(n_classes, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=output)
    return model


In [8]:
n_classes = train_generator.num_classes

model_xcep = build_model(Xception, n_classes=n_classes)
model_vgg = build_model(VGG16, n_classes=n_classes)
model_resnet = build_model(ResNet50, n_classes=n_classes)


In [ ]:
models = [model_xcep, model_vgg, model_resnet]
model_names = ['Xception', 'VGG16', 'ResNet50']

for i, model in enumerate(models):
    print(f"\nTraining {model_names[i]}...")
    model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    if train_generator.samples > 0 and val_generator.samples > 0:
         model.fit(
             train_generator,
             epochs=EPOCHS,
             validation_data=val_generator,
             callbacks=[EarlyStopping(patience=3, restore_best_weights=True)]
    )
    else: print(f"Cannot train {model_names[i]} - no data available")




Training Xception...


C:\Users\hp\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
342/342 ━━━━━━━━━━━━━━━━━━━━ 769s 2s/step - accuracy: 0.6499 - loss: 0.9170 - val_accuracy: 0.7290 - val_loss: 0.7320
Epoch 2/15
342/342 ━━━━━━━━━━━━━━━━━━━━ 735s 2s/step - accuracy: 0.7309 - loss: 0.6670 - val_accuracy: 0.7107 - val_loss: 0.7189
Epoch 3/15
342/342 ━━━━━━━━━━━━━━━━━━━━ 798s 2s/step - accuracy: 0.7326 - loss: 0.6584 - val_accuracy: 0.7382 - val_loss: 0.6471
Epoch 4/15
342/342 ━━━━━━━━━━━━━━━━━━━━ 799s 2s/step - accuracy: 0.7421 - loss: 0.6302 - val_accuracy: 0.7583 - val_loss: 0.5990
Epoch 5/15
342/342 ━━━━━━━━━━━━━━━━━━━━ 758s 2s/step - accuracy: 0.7513 - loss: 0.6196 - val_accuracy: 0.7440 - val_loss: 0.6595
Epoch 6/15
342/342 ━━━━━━━━━━━━━━━━━━━━ 728s 2s/step - accuracy: 0.7586 - loss: 0.5896 - val_accuracy: 0.7587 - val_loss: 0.6039
Epoch 7/15
342/342 ━━━━━━━━━━━━━━━━━━━━ 746s 2s/step - accuracy: 0.7510 - loss: 0.5905 - val_accuracy: 0.7554 - val_loss: 0.5688
Epoch 8/15
342/342 ━━━━━━━━━━━━━━━━━━━━ 734s 2s/step - accuracy: 0.7541 - loss: 0.5944 - val_accu

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# First, define the data_generator
data_generator = ImageDataGenerator(
    rescale=1./255,
    # You can add other augmentation parameters as needed
    # rotation_range=20,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # shear_range=0.2,
    # zoom_range=0.2,
    # horizontal_flip=True,
)

# Create the validation generator first - this was missing in the original code
val_generator = data_generator.flow_from_directory(
    r'C:\Users\hp\Documents\Data\archive',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

def get_predictions(model, generator):
    # Make sure the generator has data
    if len(generator) == 0:
        raise ValueError("Generator has no data")
    
    # Reset the generator before making predictions
    generator.reset()
    return model.predict(generator, verbose=1)

# Check if your generator has data
print(f"Number of batches in generator: {len(val_generator)}")

# Make sure your generator is properly configured
# You might need to create a new generator if the current one is empty
if len(val_generator) == 0:
    # Example of creating a new generator (adjust parameters as needed)
    val_generator = data_generator.flow_from_directory(
        r'C:\Users\hp\Documents\Data\archive',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )

# Now use the generator for predictions
test_data = val_generator
preds_xcep = get_predictions(model_xcep, test_data)
preds_vgg = get_predictions(model_vgg, test_data)
preds_resnet = get_predictions(model_resnet, test_data)

Found 13645 images belonging to 5 classes.
Number of batches in generator: 427



KeyboardInterrupt



In [ ]:
ensemble_preds = (preds_xcep + preds_vgg + preds_resnet) / 3
y_pred = np.argmax(ensemble_preds, axis=1)
y_true = test_data.classes

In [ ]:
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=list(test_data.class_indices.keys())))

cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)

In [ ]:
for model in models:
    for layer in model.layers[-20:]:
        if not isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = True

    model.compile(optimizer=Adam(learning_rate=1e-5),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    model.fit(train_generator, epochs=5, validation_data=val_generator)

In [ ]:
def plot_history(history, model_name):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(len(acc))

    plt.figure(figsize=(14, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.title(f'{model_name} - Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.title(f'{model_name} - Loss')
    plt.legend()

    plt.show()

for i in range(3):
    plot_history(histories[i], model_names[i])
